# Project 1, Part 3, Meal Related Queries



# Included Modules and Packages

Code cell containing your includes for modules and packages

In [1]:
import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

# 1.3.1 How many meals were purchased for all of AGM?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item. 

The sum of quantity in the line_items table will tell you the total meals purchased.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.


The query should return only 1 row into a Pandas dataframe and should look similar to this: 

||total_meals_purchased|
|---|---|
|0|8228284|

In [4]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT SUM(quantity) as total_meals_purchased FROM line_items

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,total_meals_purchased
0,8228284


# 1.3.2 How many meals were purchased for all of AGM by meal?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item. 

For meal_name, use the description column in the products table.

Sort by meal_name in alphabetical order.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.


The query should return 8 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||meal_name|total_meals_purchased|
|---|---|---|
|0|Brocolli Stir Fry|913984|
|...|...|...|
|7|Tilapia Piccata|687237|

In [5]:
rollback_before_flag = True
rollback_after_flag = True

query = """

WITH meal_table AS (SELECT a.description as meal_name, b.quantity FROM products a JOIN line_items b ON a.product_id = b.product_id)

SELECT meal_name, SUM(quantity) as total_meals_purchased FROM meal_table GROUP BY meal_name ORDER BY meal_name

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,meal_name,total_meals_purchased
0,Brocolli Stir Fry,913984
1,Chicken Salad,228561
2,Curry Chicken,1368884
3,Eggplant Lasagna,1599058
4,Pistachio Salmon,1828778
5,Spinach Orzo,456769
6,Teriyaki Chicken,1145013
7,Tilapia Piccata,687237


# 1.3.3 How many meals were purchased by store and by meal?

For store_name use the store's city.

Each record in the line_items table has a quantity which is the number of meals purchased for that line item.

For meal_name, use the description column in the products table.

Sort by store_name in alphabetical order, then by meal_name in alphabetical order.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.



The query should return 40 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||store_name|meal_name|total_meals_purchased|
|---|---|---|---|
|0|Berkeley|Brocolli Stir Fry|232038|
|...|...|...|...|
|39|Seattle|Tilapia Piccata|153448|

In [6]:
rollback_before_flag = True
rollback_after_flag = True

query = """

WITH meal_table_by_store AS (

SELECT c.store_name, c.quantity, d.description as meal_name 
FROM (SELECT a.city as store_name, b.product_id, b.quantity FROM stores a JOIN line_items b ON a.store_id = b.store_id) as c
JOIN products d ON c.product_id = d.product_id
) 

SELECT store_name, meal_name, SUM(quantity) as total_meals_purchased FROM meal_table_by_store
GROUP BY store_name, meal_name ORDER BY store_name, meal_name


"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,store_name,meal_name,total_meals_purchased
0,Berkeley,Brocolli Stir Fry,232038
1,Berkeley,Chicken Salad,57719
2,Berkeley,Curry Chicken,346508
3,Berkeley,Eggplant Lasagna,405637
4,Berkeley,Pistachio Salmon,464274
5,Berkeley,Spinach Orzo,115469
6,Berkeley,Teriyaki Chicken,290858
7,Berkeley,Tilapia Piccata,174252
8,Dallas,Brocolli Stir Fry,179885
9,Dallas,Chicken Salad,44756


# 1.3.4 How many meals were purchased by month?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item.

Derive the month_number (1 = January) and the month from the sale_date.

Sort by month_number.


Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.



The query should return 12 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||month_number|month|total_meals_purchased|
|---|---|---|---|
|0|1|January  |650319|
|...|...|...|...|
|11|12|December |695035|

In [17]:
rollback_before_flag = True
rollback_after_flag = True

query = """

WITH sales_by_date AS (
SELECT a.sale_date, b.quantity FROM sales a JOIN line_items b ON a.sale_id = b.sale_id 
)

SELECT EXTRACT(month FROM sale_date) AS month_number,
TO_CHAR(sale_date, 'Month') AS month,
SUM(quantity) as total_meals_purchased
FROM sales_by_date 
GROUP BY month_number, month
ORDER BY month_number

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,month_number,month,total_meals_purchased
0,1,January,3252637
1,2,February,3152564
2,3,March,3660355
3,4,April,3435052
4,5,May,3330027
5,6,June,3380074
6,7,July,3326135
7,8,August,3591531
8,9,September,2785597
9,10,October,2876508


# 1.3.5 How many meals were purchased by month and meal?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item.

Derive the month_number (1 = January) and the month from the sale_date.

For meal_name, use the description column in the products table.

Sort by month_number, then by meal_name in alphabetical order.


Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.


**Note: When a query result has a large number of rows, Pandas will only display the first 5 rows, a row with ellipses, and the last 5 rows. This is ok.**


The query should return 96 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||month_number|month|meal_name|total_meals_purchased|
|---|---|---|---|---|
|0|1|January  |Brocolli Stir Fry|72161|
|...|...|...|...|...|
|95|12|December |Tilapia Piccata|58260|

In [12]:
rollback_before_flag = True
rollback_after_flag = True

query = """

WITH sales_by_date AS (
SELECT d.description as meal_name, c.sale_date, c.quantity FROM 
(SELECT a.sale_date, b.quantity, b.product_id FROM sales a JOIN line_items b ON a.sale_id = b.sale_id) as c
JOIN products d ON c.product_id = d.product_id

)

SELECT EXTRACT(month FROM sale_date) AS month_number,
TO_CHAR(sale_date, 'Month') AS month,
meal_name,
SUM(quantity) as total_meals_purchased
FROM sales_by_date 
GROUP BY month_number, month, meal_name
ORDER BY month_number, meal_name

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,month_number,month,meal_name,total_meals_purchased
0,1,January,Brocolli Stir Fry,360924
1,1,January,Chicken Salad,90123
2,1,January,Curry Chicken,541187
3,1,January,Eggplant Lasagna,631818
4,1,January,Pistachio Salmon,720970
...,...,...,...,...
91,12,December,Eggplant Lasagna,371340
92,12,December,Pistachio Salmon,425360
93,12,December,Spinach Orzo,105496
94,12,December,Teriyaki Chicken,266347


# 1.3.6 How many meals were purchased by day of week and meal?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item.

Derive the dow (0 = Sunday) and the day_of_week from the sale_date.

For meal_name, use the description column in the products table.

Sort by dow, then by meal_name in alphabetical order

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.


The query should return 56 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||dow|day_of_week|meal_name|total_meals_purchased|
|---|---|---|---|---|
|0|0|Sunday   |Brocolli Stir Fry|172250|
|...|...|...|...|...|
|55|6|Saturday |Tilapia Piccata|135327|

In [15]:
rollback_before_flag = True
rollback_after_flag = True

query = """

WITH sales_by_date AS (
SELECT d.description as meal_name, c.sale_date, c.quantity FROM 
(SELECT a.sale_date, b.quantity, b.product_id FROM sales a JOIN line_items b ON a.sale_id = b.sale_id) as c
JOIN products d ON c.product_id = d.product_id

)

SELECT EXTRACT(DOW FROM sale_date) AS dow,
TO_CHAR(sale_date, 'Day') AS day_of_week,
meal_name,
SUM(quantity) as total_meals_purchased
FROM sales_by_date 
GROUP BY dow, day_of_week, meal_name
ORDER BY dow, meal_name

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,dow,day_of_week,meal_name,total_meals_purchased
0,0,Sunday,Brocolli Stir Fry,767460
1,0,Sunday,Chicken Salad,192077
2,0,Sunday,Curry Chicken,1149614
3,0,Sunday,Eggplant Lasagna,1343248
4,0,Sunday,Pistachio Salmon,1537853
5,0,Sunday,Spinach Orzo,383486
6,0,Sunday,Teriyaki Chicken,961561
7,0,Sunday,Tilapia Piccata,577641
8,1,Monday,Brocolli Stir Fry,545188
9,1,Monday,Chicken Salad,136507
